In [1]:
import hoomd
import hoomd.md

# Simulation contexts

Every hoomd job script **must** have at least one simulation context. The context defines how a simulation is to be executed along with the state of the system including the properties of all particles, forces, integrators, etc... Create a simulation context with ``hoomd.context.initialize``.

In [2]:
hoomd.context.initialize('--mode=cpu');

HOOMD-blue v2.1.5 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 03/09/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


## Execution context

The **first** simulation context created in the simulation also creates the execution context. Once set, the execution context cannot be changed without restarting python. Note how the above context is running on the CPU and how the following context ignores the request to change execution options.

In [3]:
hoomd.context.initialize('--mode=gpu');

*Warning*: Ignoring new options, cannot change execution mode after initialization.


The execution context defines the device the simulation will execute on (i.e. CPU or GPU), how many MPI ranks to run in parallel, GPU device options, etc... Control these parameters with command line options. For a script run on the command line, use ``hoomd.context.initialize()`` and pass the options in from the command line ``python script.py [options]``. In a jupyter notebook, specify the command line options in the string argument to ``hoomd.context.initialize``. 

## Simulation context

Each time you call it, ``hoomd.context.initialize`` creates a new simulation context and makes it active. You can execute multiple simulation runs in a script by using different contexts. 

Here are some short MD simulations. Notice how the 2nd simulation runs fewer steps per second because it has more particles.

In [4]:
sim1 = hoomd.context.initialize('');
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);
# setup forces, integrators and run simulation 1
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.001);
hoomd.md.integrate.brownian(group=all, kT=0.1, seed=987);
hoomd.run(2e4);

*Warning*: Ignoring new options, cannot change execution mode after initialization.


notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:02 | Step 20000 / 20000 | TPS 8231.4 | ETA 00:00:00
Average TPS: 8213.56
---------
-- Neighborlist stats:
694 normal updates / 201 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 53 / n_neigh_avg: 22.128
shortest rebuild period: 11
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 0 / n_max: 58 / n_avg: 15.625
** run complete **


In [5]:
sim2 = hoomd.context.initialize('');
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=10);
# setup forces, integrators and run simulation 2
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.001);
hoomd.md.integrate.brownian(group=all, kT=0.2, seed=10);
hoomd.run(2e3);

notice(2): Group "all" created containing 1000 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 1000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **


*Warning*: Ignoring new options, cannot change execution mode after initialization.


Time 00:00:02 | Step 2000 / 2000 | TPS 871.58 | ETA 00:00:00
Average TPS: 869.349
---------
-- Neighborlist stats:
265 normal updates / 21 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 26 / n_neigh_avg: 9.731
shortest rebuild period: 5
-- Cell list stats:
Dimension: 5, 5, 5
n_min    : 1 / n_max: 18 / n_avg: 8
** run complete **


## Switching between contexts

The above example showed how to run multiple simulations in one script, one after another. You can also save contexts in variables and swap between them. All hoomd commands apply to the current context only.

Activate simulation 1 and continue running it for more steps.

In [6]:
with sim1:
    hoomd.run(2e4);

** starting run **
Time 00:00:07 | Step 40000 / 40000 | TPS 9049.7 | ETA 00:00:00
Average TPS: 9011.72
---------
-- Neighborlist stats:
556 normal updates / 201 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 54 / n_neigh_avg: 23.888
shortest rebuild period: 12
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 0 / n_max: 61 / n_avg: 15.625
** run complete **


Activate simulation 2 and continue running it for more steps.

In [7]:
with sim2:
    hoomd.run(2e3);

** starting run **
Time 00:00:06 | Step 4000 / 4000 | TPS 1078.75 | ETA 00:00:00
Average TPS: 1073.88
---------
-- Neighborlist stats:
248 normal updates / 21 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 52 / n_neigh_avg: 12.267
shortest rebuild period: 5
-- Cell list stats:
Dimension: 5, 5, 5
n_min    : 0 / n_max: 27 / n_avg: 8
** run complete **
